# Venue 72 - Lascar loggers

Venue 72 was able to deploy a bunch of Lascars at once - commercial USB-stick loggers that record temperature, and relative humidity if you pay more.  We can't always support this, but it can be useful for understanding balance and circulation problems and thinking about which rooms are really struggling with heat loss.

The heating timings are taken from the timeswitch in February but apart from Fridays and some off times probably haven't changed.  We've only marked the cafe opening times as we aren't sure of the building use otherwise. 

You can get a line to disappear by clicking on it in the legend.  



In [ ]:
# Using plotly.express

venue_data_dir = "./venue-72/" 
venue_temp_data_dir = venue_data_dir + "/" 
weather_data_dir = "./weather_data/"


temp_filenames = ["balcony_left.csv","balcony_right.csv","balcony_middle.csv", "platform_left_pillar.csv","platform_right_pillar.csv", "under_piano.csv"]
location_labels= ["balcony_left","balcony_right","balcony_middle", "platform_left_pillar","platform_right_pillar", "under_piano"]
main_location_label = "under_piano" #"platform_left_pillar" # used to plot against weather data.
use_diary_filepath = venue_data_dir + "use_diary.csv"
heating_timings_filepath =  venue_data_dir + "heating_timings.csv"
weather_filepath = weather_data_dir + "gcht2024_weather_data.csv"


# import ipywidgets as widgets
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go   
import os as os
#from IPython.display import display

# now the homebuilt monitor

plot_height = 800
plot_width = 1000

list_of_temp_traces = []
list_of_rh_traces = []
list_of_traces_for_weather_comparison = []
dfHeatTimes = pd.DataFrame()
dfUseDiary = pd.DataFrame()

if os.path.isfile(use_diary_filepath):
    dfUseDiary = pd.read_csv(use_diary_filepath)
    dfUseDiary = dfUseDiary.fillna('')
    dfUseDiary['annotation_position'] = dfUseDiary['annotation_position'].replace(to_replace="", value="top left")
    dfUseDiary['start'] = pd.to_datetime(dfUseDiary['startUse'])
    dfUseDiary['end'] = pd.to_datetime(dfUseDiary['endUse'])
    #print(dfUseDiary)

if os.path.isfile(heating_timings_filepath):
    dfHeatTimes = pd.read_csv(heating_timings_filepath)
    dfHeatTimes = dfHeatTimes.fillna('')
    dfHeatTimes['start'] = pd.to_datetime(dfHeatTimes['startHeat'])
    dfHeatTimes['end'] = pd.to_datetime(dfHeatTimes['endHeat'])


for temp_filename,location_label in zip(temp_filenames,location_labels):
    if (location_label== "under_piano"):
        df = pd.read_csv(venue_temp_data_dir + temp_filename,encoding='latin-1',usecols=[0,2,3],header=1,names=['timestamp', 'temperature','rh'])
        df["timestamp"] = pd.to_datetime(df['timestamp'])
        df = df.fillna('')
    else:
        df = pd.read_csv(venue_temp_data_dir + temp_filename,encoding='latin-1',usecols=[1,2,3], header=1, names=['timestamp', 'temperature','rh'])
        df["timestamp"] = pd.to_datetime(df['timestamp'])
        df = df.fillna('')
    
    #range_min = df['timestamp'].min()
    #range_max = df['timestamp'].max()
    #print(range_min, " to ", range_max) # this just uses the range of the last file, not ideal
    
    
    trace = go.Scatter(customdata=df, 
                        y=df['temperature'], 
                        x = df['timestamp'], 
                        mode='lines', 
                        hoverinfo='all', 
                        name=location_label,
                        )
    list_of_temp_traces.append(trace)

    # take one of the less wild traces and add it to plot it against the weather.
    if (location_label == main_location_label):
        list_of_traces_for_weather_comparison.append(trace)
    
    trace = go.Scatter(customdata=df, 
                        y=df['rh'], 
                        x = df['timestamp'], 
                        mode='lines', 
                        hoverinfo='all', 
                        name=location_label,
                        )
    list_of_rh_traces.append(trace)

# create a trace for the weather data

df = pd.read_csv(weather_filepath,encoding='latin-1',usecols=[0,1,2], header=1, names=[ 'time','temperature','windspeed'])
df["timestamp"] = pd.to_datetime(df['time'])
df = df.fillna('')
df['windspeed'] = pd.to_numeric(df['windspeed'], downcast='float', errors='coerce')
    
trace = go.Scatter(customdata=df, 
                        y=df['temperature'], 
                        x = df['timestamp'], 
                        mode='lines', 
                        hoverinfo='all', 
                        name="outside",
                        )

list_of_traces_for_weather_comparison.append(trace)

trace = go.Scatter(customdata=df, 
                        y=df['windspeed']/10, 
                        x = df['timestamp'], 
                        mode='lines', 
                        hoverinfo='all', 
                        name="wind speed (mph)/10",
                        )

list_of_traces_for_weather_comparison.append(trace)

temp_g = go.FigureWidget(data=list_of_temp_traces,
                    layout = go.Layout(
                        yaxis=dict(range=[10,25])
                    ))

rh_g = go.FigureWidget(data=list_of_rh_traces,
                    layout = go.Layout(
                        yaxis=dict(range=[0,100])
                    ))

weather_g = go.FigureWidget(data=list_of_traces_for_weather_comparison,
                    layout = go.Layout(
                        yaxis=dict(range=[-6,25])
                    )) 

# example syntax for two plots on same x-axis - I'd like to show the boiler temperature in
# parallel - but havne't had time to sort the syntax.
#fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

# for i, col in enumerate(cols, start=1):
#     fig.add_trace(go.Scatter(x=df[col].index, y=df[col].values), row=i, col=1)

temp_fig = go.Figure(temp_g)
temp_fig.update_layout(showlegend=True, 
              autosize = True, 
              title= "Temperature (deg C))",
              width=plot_width, 
              height=plot_height,
)

rh_fig = go.Figure(rh_g)
rh_fig.update_layout(showlegend=True, 
              autosize = True, 
              title= "Relative Humidity (%RH)",
              width=plot_width, 
              height=plot_height,
)


weather_fig = go.Figure(weather_g)
weather_fig.update_layout(showlegend=True, 
              autosize = True, 
              title= "Indoor vs outdoor temperature from weather station (deg C))",
              width=plot_width, 
              height=plot_height,
)

temp_fig.update_layout(
    hovermode='x unified',
   # range=[range_min, range_max],
    hoverlabel=dict(
        bgcolor="white",
        # font_size=16,
        font_family="Rockwell"
    )
)

#Add range slider
temp_fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(
                     label="All",
                     step="all"
                     ),
                                dict(count=1,
                     label="Hour",
                     step="hour",
                     stepmode="todate"),
                dict(count=1,
                     label="Day",
                     step="day",
                     stepmode="backward"),
                dict(count=7,
                     label="Week",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="Year",
                     step="year",
                     stepmode="backward")
            ])
        ),
        rangeslider=dict(
            visible=True,
        ),
        type="date"
    )
)
 

for ind in dfUseDiary.index:
    temp_fig.add_vrect(x0=dfUseDiary['start'][ind], x1=dfUseDiary['end'][ind],
                annotation_text=dfUseDiary['label'][ind], 
                annotation_position=dfUseDiary['annotation_position'][ind],
                annotation=dict(font_size=14, 
                font_family="Times New Roman"),
                fillcolor="green", 
                opacity=0.25, 
                line_width=0)
        

for ind in dfHeatTimes.index:
    temp_fig.add_vrect(x0=dfHeatTimes['start'][ind], x1=dfHeatTimes['end'][ind])

#fig.add_hline(y=16, annotation_text='16C - usual minimum for children', annotation_font_color="blue", line_color='red', layer='above', line_dash='dash')
# fig.update_yaxes(range = [-5, dfCollatedDataSet['temperature'].max()+5])
temp_fig.show()

rh_fig.update_layout(
    hovermode='x unified',
   # range=[range_min, range_max],
    hoverlabel=dict(
        bgcolor="white",
        # font_size=16,
        font_family="Rockwell"
    )
)

#Add range slider
rh_fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(
                     label="All",
                     step="all"
                     ),
                                dict(count=1,
                     label="Hour",
                     step="hour",
                     stepmode="todate"),
                dict(count=1,
                     label="Day",
                     step="day",
                     stepmode="backward"),
                dict(count=7,
                     label="Week",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="Year",
                     step="year",
                     stepmode="backward")
            ])
        ),
        rangeslider=dict(
            visible=True,
        ),
        type="date"
    )
)


#fig.add_hline(y=16, annotation_text='16C - usual minimum for children', annotation_font_color="blue", line_color='red', layer='above', line_dash='dash')
# fig.update_yaxes(range = [-5, dfCollatedDataSet['temperature'].max()+5])
rh_fig.show()

weather_fig.update_layout(
    hovermode='x unified',
   # range=[range_min, range_max],
    hoverlabel=dict(
        bgcolor="white",
        # font_size=16,
        font_family="Rockwell"
    )
)

#Add range slider
weather_fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(
                     label="All",
                     step="all"
                     ),
                                dict(count=1,
                     label="Hour",
                     step="hour",
                     stepmode="todate"),
                dict(count=1,
                     label="Day",
                     step="day",
                     stepmode="backward"),
                dict(count=7,
                     label="Week",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="Year",
                     step="year",
                     stepmode="backward")
            ])
        ),
        rangeslider=dict(
            visible=True,
        ),
        type="date"
    )
)


for ind in dfHeatTimes.index:
    weather_fig.add_vrect(x0=dfHeatTimes['start'][ind], x1=dfHeatTimes['end'][ind])

#fig.add_hline(y=16, annotation_text='16C - usual minimum for children', annotation_font_color="blue", line_color='red', layer='above', line_dash='dash')
# fig.update_yaxes(range = [-5, dfCollatedDataSet['temperature'].max()+5])

weather_fig.show()
